In [1]:
import pandas as pd
import math
import numpy as np
import statsmodels.api as sm
import matplotlib.pyplot as plt
from scipy import stats

import datetime

from statsmodels.tsa.regime_switching.markov_autoregression import MarkovAutoregression
from statsmodels.tsa.regime_switching.markov_regression import MarkovRegression

In [2]:
#load data
data = pd.read_csv('data/milk.csv')
print data.columns

#include incumbent to indentify things

Index([u'VENDOR', u'WW', u'WC', u'LFW', u'LFC', u'WIN', u'SYSTEM', u'YEAR',
       u'MONTH', u'DAY', u'FMOZONE', u'ESC', u'COOLER', u'QLFC', u'QLFW',
       u'QWW', u'QWC', u'ESTQTY', u'DEL', u'MILES', u'NUMSCHL', u'NUMWIN',
       u'POPUL', u'ADJPOP', u'NUM', u'GAS', u'FMO', u'INC'],
      dtype='object')


In [7]:
reg1 = data[['WW','FMO','INC','ESC','ESTQTY','DEL','COOLER']].dropna()
reg1 = sm.add_constant(reg1)
reg1 = reg1[reg1['WW']<.4]

reg1[['WW','FMO']] = np.log(reg1[['WW','FMO']])
model = sm.OLS(reg1['WW'],reg1[['const','FMO','INC','ESC','COOLER']])
result = model.fit()
print result.summary()
print result.summary().as_latex()

                            OLS Regression Results                            
Dep. Variable:                     WW   R-squared:                       0.098
Model:                            OLS   Adj. R-squared:                  0.097
Method:                 Least Squares   F-statistic:                     75.08
Date:                Sat, 27 Apr 2019   Prob (F-statistic):           1.72e-60
Time:                        16:13:52   Log-Likelihood:                 2714.5
No. Observations:                2764   AIC:                            -5419.
Df Residuals:                    2759   BIC:                            -5389.
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         -2.8564      0.079    -36.192      0.0

In [6]:
model = MarkovRegression(endog=reg1['WW'], exog=reg1[['const','FMO','INC','ESC','COOLER']],
                         k_regimes=2, order=0, trend='nc',switching_exog=[False, False, True, False, False])
result = model.fit()
print result.summary()

/home/erichschulman/anaconda2/lib/python2.7/site-packages/statsmodels/tsa/base/tsa_model.py:221: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  ' ignored when e.g. forecasting.', ValueWarning)


                        Markov Switching Model Results                        
Dep. Variable:                     WW   No. Observations:                 2764
Model:               MarkovRegression   Log Likelihood                2756.871
Date:                Sat, 27 Apr 2019   AIC                          -5495.742
Time:                        15:54:15   BIC                          -5442.422
Sample:                             0   HQIC                         -5476.483
                               - 2764                                         
Covariance Type:               approx                                         
                             Regime 0 parameters                              
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
INC            0.4644      0.047      9.871      0.000       0.372       0.557
                             Regime 1 parameters    

In [ ]:
#ESC, COOLER ,MILES, GAS, DEL, ESTQTY
reg1 = data[['WW', 'FMO','ESC', 'NUMSCHL','MILES','COOLER', 'GAS','DEL','ESTQTY']].dropna()

#make a little better
reg1['NUMSCHL'] = np.maximum(reg1['NUMSCHL'],1)
reg1['NOSTOP'] = reg1['NUMSCHL']*reg1['DEL']
reg1['MILES'] =reg1['MILES']+1
reg1 = reg1[reg1['WW']<=.3]

#log scale
reg1[['WW', 'FMO', 'MILES', 'GAS','NOSTOP', 'ESTQTY']] = np.log(
                                            reg1[['WW', 'FMO', 'MILES', 'GAS', 'NOSTOP', 'ESTQTY']])

print reg1.mean()
